In [ ]:
import json
import pandas as pd
from collections import defaultdict

# BASSE dataset

Procesa un archivo JSONL de resúmenes y devuelve una lista de objetos (diccionarios).
Cada objeto contiene el índice, el documento original y los resúmenes de varios modelos.
Además, se puede convertir la lista de objetos en un DataFrame de Pandas para un análisis más fácil.

In [ ]:
def get_list_of_objects_from_basse_dataset(filepath):
    """
    Procesa un archivo JSONL de resúmenes y devuelve una lista de objetos (diccionarios).

    Args:
        filepath (str): La ruta al archivo .jsonl.

    Returns:
        list: Una lista de diccionarios, donde cada diccionario representa un objeto JSON procesado.
    """
    data_list = []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            try:
                json_object = json.loads(line.strip())

                idx = json_object.get('idx')
                round_val = json_object.get('round')
                original_document = json_object.get('original_document')

                model_summaries = json_object.get('model_summaries', {})

                claude_summ = model_summaries.get('claude-5w1h', {}).get('summ')
                commandr_summ = model_summaries.get('commandr-5w1h', {}).get('summ')
                gpt4o_summ = model_summaries.get('gpt4o-5w1h', {}).get('summ')
                reka_summ = model_summaries.get('reka-5w1h', {}).get('summ')
                llama3_summ = model_summaries.get('llama3-5w1h', {}).get('summ')

                data_list.append({
                    'idx': idx,
                    'round': round_val,
                    'original_document': original_document,
                    'claude-5w1h_summ': claude_summ,
                    'commandr-5w1h_summ': commandr_summ,
                    'gpt4o-5w1h_summ': gpt4o_summ,
                    'reka-5w1h_summ': reka_summ,
                    'llama3-5w1h_summ': llama3_summ
                })
            except json.JSONDecodeError:
                print(f"Advertencia (Línea {line_number}): Se omitió una línea (resúmenes) debido a un error de decodificación JSON: {line.strip()}")
            except Exception as e:
                print(f"Advertencia (Línea {line_number}): Se omitió una línea (resúmenes) debido a un error inesperado ({e}): {line.strip()}")

    return data_list

Procesar el archivo JSONL de resúmenes y generar una lista de diccionarios.


In [ ]:
# Procesar el archivo JSONL de resúmenes
jsonl_file_path_summaries = 'data/basse/BASSE.jsonl'
list_of_summary_objects = get_list_of_objects_from_basse_dataset(jsonl_file_path_summaries)

# Ahora 'list_of_summary_objects' es una lista de diccionarios.
if list_of_summary_objects:
    print("Primer objeto de la lista de resúmenes:")
    print(json.dumps(list_of_summary_objects[0], indent=2, ensure_ascii=False))

Convertir la lista de diccionarios en un DataFrame de Pandas (opcional)

In [ ]:
# Para convertir la lista de diccionarios en un DataFrame de Pandas:
df_summaries = pd.DataFrame(list_of_summary_objects)
print("\nDataFrame creado desde la lista de resúmenes:")
df_summaries

# Flares dataset
Similar al anterior, pero para un archivo JSON que contiene una lista de objetos, cada uno con un id, text, y una lista de tags.
Cada tag tiene un campo '5W1H_Label', 'Reliability_Label' y 'Tag_Text'.

In [ ]:
def process_single_object_tags_nested(json_object):
    """
    Procesa un único objeto JSON y anida las etiquetas procesadas (con enumeración)
    dentro de una lista en el objeto resultante.
    """
    processed_object = {
        'Id': json_object.get('Id'),
        'Text': json_object.get('Text'),
        'Processed_Tags': [] # Lista para las etiquetas procesadas
    }

    tags = json_object.get('Tags', [])
    w5h1_label_counts = defaultdict(int) # Contador para enumerar

    if tags:
        for tag_item in tags:
            original_w5h1_label = tag_item.get('5W1H_Label')
            reliability_label = tag_item.get('Reliability_Label')
            tag_text = tag_item.get('Tag_Text')

            enumerated_tag_id = None
            if original_w5h1_label:
                w5h1_label_counts[original_w5h1_label] += 1
                current_count = w5h1_label_counts[original_w5h1_label]
                enumerated_tag_id = f"{original_w5h1_label}_{current_count}"

            processed_object['Processed_Tags'].append({
                'Original_5W1H_Label': original_w5h1_label, # Mantenemos la original
                'Enumerated_Tag_Id': enumerated_tag_id, # Ej: WHO_1, WHAT_2
                'Reliability_Label': reliability_label,
                'Tag_Text': tag_text
            })

    return processed_object

def get_list_of_objects_from_tags_jsonl_nested(filepath):
    """
    Procesa un archivo JSONL (con estructura de Tags) y devuelve una lista de objetos,
    donde cada objeto contiene sus etiquetas procesadas de forma anidada.

    Args:
        filepath (str): La ruta al archivo .jsonl.

    Returns:
        list: Una lista de diccionarios.
    """
    all_processed_objects = []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            try:
                json_object = json.loads(line.strip())
                processed_data = process_single_object_tags_nested(json_object)
                all_processed_objects.append(processed_data)
            except json.JSONDecodeError:
                print(f"Advertencia (Línea {line_number}): Se omitió una línea (etiquetas) debido a un error de decodificación JSON: {line.strip()}")
            except Exception as e:
                print(f"Advertencia (Línea {line_number}): Se omitió una línea (etiquetas) debido a un error inesperado ({e}): {line.strip()}")

    return all_processed_objects

Procesar el archivo JSONL de etiquetas y generar una lista de diccionarios.

In [ ]:
jsonl_file_path_tags_nested = 'data/flares/5w1h_subtask_1_trial.json'
list_of_tagged_objects_nested = get_list_of_objects_from_tags_jsonl_nested(jsonl_file_path_tags_nested)

# Ahora 'list_of_tagged_objects_nested' es una lista de diccionarios,
# y cada diccionario tiene una clave 'Processed_Tags' con una lista de etiquetas.
if list_of_tagged_objects_nested:
    print("Primer objeto de la lista de etiquetas (estructura anidada):")
    print(json.dumps(list_of_tagged_objects_nested[0], indent=2, ensure_ascii=False))

Para crear un DataFrame principal (las etiquetas estarán como listas en una columna):

In [ ]:
df_tags_main = pd.DataFrame(list_of_tagged_objects_nested)
print("\nDataFrame principal creado desde la lista de etiquetas (anidado):")
df_tags_main

Para "desanidar" las etiquetas en filas separadas (formato largo):


In [ ]:
if not df_tags_main.empty and 'Processed_Tags' in df_tags_main.columns:
    df_tags_exploded = df_tags_main.explode('Processed_Tags')
      # Normalizar la columna que ahora contiene diccionarios de etiquetas
    df_tags_normalized = pd.json_normalize(df_tags_exploded['Processed_Tags'])
      # Unir con las columnas originales (Id, Text) del DataFrame explotado
    df_tags_final_long = pd.concat(
        [df_tags_exploded[['Id', 'Text']].reset_index(drop=True), df_tags_normalized.reset_index(drop=True)],
        axis=1
    )
print("\nDataFrame en formato largo (cada etiqueta es una fila):")
df_tags_final_long